# Простой классификатор изображений

Этот ноутбук демонстрирует, как классифицировать изображения с использованием предварительно обученной нейронной сети.

**Чему вы научитесь:**
- Как загрузить и использовать предварительно обученную модель
- Предварительная обработка изображений
- Выполнение предсказаний на изображениях
- Понимание уровней уверенности

**Пример использования:** Определение объектов на изображениях (например, "кот", "собака", "машина" и т.д.)

---


## Шаг 1: Импорт необходимых библиотек

Давайте импортируем инструменты, которые нам понадобятся. Не переживайте, если пока не понимаете их всех!


In [ ]:
# Core libraries
import numpy as np
from PIL import Image
import requests
from io import BytesIO

# TensorFlow for deep learning
try:
    import tensorflow as tf
    from tensorflow.keras.applications import MobileNetV2
    from tensorflow.keras.applications.mobilenet_v2 import preprocess_input, decode_predictions
    print("✅ TensorFlow loaded successfully!")
    print(f"   Version: {tf.__version__}")
except ImportError:
    print("❌ Please install TensorFlow: pip install tensorflow")

## Шаг 2: Загрузка предварительно обученной модели

Мы будем использовать **MobileNetV2**, нейронную сеть, уже обученную на миллионах изображений.

Это называется **Трансферное обучение** – использование модели, которую обучил кто-то другой!


In [ ]:
print("📦 Loading pre-trained MobileNetV2 model...")
print("   This may take a minute on first run (downloading weights)...")

# Load the model
# include_top=True means we use the classification layer
# weights='imagenet' means it was trained on ImageNet dataset
model = MobileNetV2(weights='imagenet', include_top=True)

print("✅ Model loaded!")
print(f"   The model can recognize 1000 different object categories")

## Шаг 3: Вспомогательные функции

Давайте создадим функции для загрузки и подготовки изображений для нашей модели.


In [ ]:
def load_image_from_url(url):
    """
    Load an image from a URL.
    
    Args:
        url: Web address of the image
        
    Returns:
        PIL Image object
    """
    response = requests.get(url)
    img = Image.open(BytesIO(response.content))
    return img


def prepare_image(img):
    """
    Prepare an image for the model.
    
    Steps:
    1. Resize to 224x224 (model's expected size)
    2. Convert to array
    3. Add batch dimension
    4. Preprocess for MobileNetV2
    
    Args:
        img: PIL Image
        
    Returns:
        Preprocessed image array
    """
    # Resize to 224x224 pixels
    img = img.resize((224, 224))
    
    # Convert to numpy array
    img_array = np.array(img)
    
    # Add batch dimension (model expects multiple images)
    img_array = np.expand_dims(img_array, axis=0)
    
    # Preprocess for MobileNetV2
    img_array = preprocess_input(img_array)
    
    return img_array


def classify_image(img):
    """
    Classify an image and return top predictions.
    
    Args:
        img: PIL Image
        
    Returns:
        List of (class_name, confidence) tuples
    """
    # Prepare the image
    img_array = prepare_image(img)
    
    # Make prediction
    predictions = model.predict(img_array, verbose=0)
    
    # Decode predictions to human-readable labels
    # top=5 means we get the top 5 most likely classes
    decoded = decode_predictions(predictions, top=5)[0]
    
    # Convert to simpler format
    results = [(label, float(confidence)) for (_, label, confidence) in decoded]
    
    return results


print("✅ Helper functions ready!")

## Шаг 4: Тестирование на примерах изображений

Давайте попробуем классифицировать несколько изображений из интернета!


In [ ]:
# Sample images to classify
# These are from Unsplash (free stock photos)
test_images = [
    {
        "url": "https://images.unsplash.com/photo-1514888286974-6c03e2ca1dba?w=400",
        "description": "A cat"
    },
    {
        "url": "https://images.unsplash.com/photo-1552053831-71594a27632d?w=400",
        "description": "A dog"
    },
    {
        "url": "https://images.unsplash.com/photo-1511919884226-fd3cad34687c?w=400",
        "description": "A car"
    },
]

print(f"🧪 Testing on {len(test_images)} images...")
print("=" * 70)

### Классифицируйте каждое изображение


In [ ]:
for i, img_data in enumerate(test_images, 1):
    print(f"\n📸 Image {i}: {img_data['description']}")
    print("-" * 70)
    
    try:
        # Load image
        img = load_image_from_url(img_data['url'])
        
        # Display image
        display(img.resize((200, 200)))  # Show smaller version
        
        # Classify
        results = classify_image(img)
        
        # Show predictions
        print("\n🎯 Top 5 Predictions:")
        for rank, (label, confidence) in enumerate(results, 1):
            # Create a visual bar
            bar_length = int(confidence * 50)
            bar = "█" * bar_length
            
            print(f"  {rank}. {label:20s} {confidence*100:5.2f}% {bar}")
        
    except Exception as e:
        print(f"❌ Error: {e}")

print("\n" + "=" * 70)

## Шаг 5: Попробуйте свои изображения!

Замените URL ниже на любой URL изображения, которое вы хотите классифицировать.


In [ ]:
# Try your own image!
# Replace this URL with any image URL
custom_image_url = "https://images.unsplash.com/photo-1472491235688-bdc81a63246e?w=400"  # A flower

print("🖼️  Classifying your custom image...")
print("=" * 70)

try:
    # Load and show image
    img = load_image_from_url(custom_image_url)
    display(img.resize((300, 300)))
    
    # Classify
    results = classify_image(img)
    
    # Show results
    print("\n🎯 Top 5 Predictions:")
    print("-" * 70)
    for rank, (label, confidence) in enumerate(results, 1):
        bar_length = int(confidence * 50)
        bar = "█" * bar_length
        print(f"  {rank}. {label:20s} {confidence*100:5.2f}% {bar}")
    
    # Highlight top prediction
    top_label, top_confidence = results[0]
    print("\n" + "=" * 70)
    print(f"\n🏆 Best guess: {top_label} ({top_confidence*100:.2f}% confident)")
    
except Exception as e:
    print(f"❌ Error: {e}")
    print("   Make sure the URL points to a valid image!")

## 💡 Что только что произошло?

1. **Мы загрузили предварительно обученную модель** - MobileNetV2 была обучена на миллионах изображений.
2. **Мы обработали изображения** - Изменили размер и подготовили их для модели.
3. **Модель сделала предсказания** - Она выдала вероятности для 1000 классов объектов.
4. **Мы декодировали результаты** - Преобразовали числа в понятные человеку метки.

### Понимание уровней уверенности

- **90-100%**: Очень уверенно (почти наверняка правильно)
- **70-90%**: Уверенно (скорее всего правильно)
- **50-70%**: Умеренно уверенно (возможно правильно)
- **Менее 50%**: Не очень уверенно (неопределенно)

### Почему предсказания могут быть неверными?

- **Необычный угол или освещение** - Модель обучалась на типичных фотографиях.
- **Несколько объектов** - Модель ожидает один основной объект.
- **Редкие объекты** - Модель знает только 1000 категорий.
- **Низкое качество изображения** - Размытые или пикселизированные изображения сложнее распознать.

---


## 🚀 Следующие шаги

1. **Попробуйте разные изображения:**
   - Найдите изображения на [Unsplash](https://unsplash.com)
   - Щелкните правой кнопкой мыши → «Копировать адрес изображения», чтобы получить URL

2. **Экспериментируйте:**
   - Что произойдет с абстрактным искусством?
   - Может ли модель распознавать объекты под разными углами?
   - Как она справляется с несколькими объектами?

3. **Узнайте больше:**
   - Изучите [уроки по компьютерному зрению](../lessons/4-ComputerVision/README.md)
   - Научитесь обучать собственный классификатор изображений
   - Поймите, как работают CNN (сверточные нейронные сети)

---

## 🎉 Поздравляем!

Вы только что создали классификатор изображений, используя передовую нейронную сеть!

Такая же техника используется для:
- Google Photos (организация ваших фотографий)
- Автономных автомобилей (распознавание объектов)
- Медицинской диагностики (анализ рентгеновских снимков)
- Контроля качества (обнаружение дефектов)

Продолжайте исследовать и учиться! 🚀



---

**Отказ от ответственности**:  
Этот документ был переведен с помощью сервиса автоматического перевода [Co-op Translator](https://github.com/Azure/co-op-translator). Несмотря на наши усилия обеспечить точность, автоматические переводы могут содержать ошибки или неточности. Оригинальный документ на его родном языке следует считать авторитетным источником. Для получения критически важной информации рекомендуется профессиональный перевод человеком. Мы не несем ответственности за любые недоразумения или неправильные интерпретации, возникшие в результате использования данного перевода.
